# Example: Import district heating network from GeoJSON files
============================================================

This example demonstrates how to import a district heating network
from three GeoJSON files representing network topology, buildings,
and supply points.


In [1]:

import os
from uesgraphs import UESGraph
from uesgraphs.examples import e1_readme_example as e1

#Set workspace
workspace = e1.workspace_example("e15")


# Get path to example data
uesgraphs_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(uesgraphs_dir, 'data','examples', 'e15_geojson')

# Create graph and import GeoJSON
graph = UESGraph()
graph.from_geojson(
    network_path=os.path.join(data_dir, 'network.geojson'),
    buildings_path=os.path.join(data_dir, 'buildings.geojson'),
    supply_path=os.path.join(data_dir, 'supply.geojson'),
    name='simple_district',
    save_path=workspace,
    generate_visualizations=True
)


print(f"============ Uesgraphs Import from GeoJSON successful! ============")
print(f"Network nodes: {graph.number_of_nodes('heating')}")
print(f"Buildings: {graph.number_of_nodes('building')}")
print(f"Total length: {graph.network_length:.2f} m")
print(f"Visualizations of network generation can be found in: {os.path.join(workspace, 'visualizations')}")

graph.to_json(os.path.join(workspace, 'simple_district_graph.json'))

Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_123812.log


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.
Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\1_basic_uesgraph.pdf
Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_123813.log
Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\2_with_supply.pdf
Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_123813.log


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.
Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\3_with_bldg.pdf
Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_123813.log
Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\4_coords_transformed.pdf
Total network length (m):  426.27
============ Uesgraphs Import from GeoJSON successful! ============
Network nodes: 8
Buildings: 5
Total length: 426.27 m
Visualizations of network generation can be found in: E:\rka_lko\git\uesgraphs\workspace\e15\visualizations


'E:\\rka_lko\\git\\uesgraphs\\workspace\\e15\\simple_district_graph.json\\nodes.json'

In [2]:
graph.nodes

NodeView((1002, 1004, 1006, 1007, 1010, 1014, 1018, 1022, 1025, 1026, 1027, 1028, 1029))

In [3]:

import pandas as pd
import os
from pathlib import Path
import logging


def load_component_parameters(excel_path, component_type):
    """
    Load component parameters from an Excel file.
    
    Reads a specific sheet from an Excel file and returns parameters as a dictionary.
    Expected Excel structure:
    - Column A: Parameter (parameter names)
    - Column B: Value (parameter values)
    
    Parameters
    ----------
    excel_path : str or Path
        Path to the Excel file containing component parameters
    component_type : str
        Type of component, must be one of: 'pipes', 'supply', 'demands', 'simulation'
        This determines which sheet to read from the Excel file
        
    Returns
    -------
    dict
        Dictionary with parameter names as keys and their values
        Returns empty dict if sheet not found
        
    Raises
    ------
    FileNotFoundError
        If the Excel file does not exist
    ValueError
        If the component_type is not valid or Excel structure is incorrect
        
    Examples
    --------
    >>> params = load_component_parameters('parameters.xlsx', 'pipes')
    >>> print(params['dp_nominal'])
    0.10
    """
    # Validate component type
    valid_types = ['Pipes', 'Supply', 'Demands', 'Simulation']
    if component_type not in valid_types:
        raise ValueError(
            f"Invalid component_type '{component_type}'. "
            f"Must be one of: {', '.join(valid_types)}"
        )
    
    # Check if file exists
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Excel file not found: {excel_path}")
    
    try:
        # Read the specific sheet
        df = pd.read_excel(excel_path, sheet_name=component_type)
        
        # Validate Excel structure
        if len(df.columns) < 2:
            raise ValueError(
                f"Excel sheet '{component_type}' must have at least 2 columns "
                "(Parameter and Value)"
            )
        
        # Check if first row contains expected column names
        if 'Parameter' not in df.columns and 'parameter' not in df.columns.str.lower():
            # Assume first two columns are Parameter and Value
            df.columns = ['Parameter', 'Value'] + list(df.columns[2:])
        
        # Create dictionary from Parameter and Value columns
        # Drop rows where Parameter is NaN
        df_clean = df[['Parameter', 'Value']].dropna(subset=['Parameter'])
        
        # Convert to dictionary
        param_dict = dict(zip(df_clean['Parameter'], df_clean['Value']))
        
        return param_dict
        
    except ValueError as e:
        if "Worksheet named" in str(e):
            # Sheet doesn't exist
            logging.warning(
                f"Sheet '{component_type}' not found in {excel_path}. "
                f"Returning empty dictionary."
            )
            return {}
        else:
            raise
    
    except Exception as e:
        raise Exception(
            f"Error reading Excel file {excel_path}, sheet '{component_type}': {e}"
        )

In [4]:
load_component_parameters(r"C:\Users\rka-lko\Downloads\uesgraphs_parameters_template.xlsx", "Pipes")

{'template': 'PlugFlowPipeEmbedded',
 'template_path': 'E:\\rka_lko\\git\\uesgraphs\\uesgraphs\\data\\templates\\network\\pipe\\AixLib_Fluid_DistrictHeatingCooling_Pipes_PlugFlowPipeEmbedded.mako',
 'dIns': 0.04,
 'kIns': 0.03,
 'fac': 1.5,
 'roughness': 0,
 'ground_depth': 1,
 'static_pipe_length': nan,
 'allowFlowReversal': 'TRUE'}

In [5]:
from mako.template import Template
def parse_template_parameters(template_path, logger=None):
    """
    Parse a Mako template file to extract required and optional parameters.
    
    Reads a .mako template file and extracts the parameter lists defined in:
    - get_main_parameters: Required parameters (must be provided)
    - get_aux_parameters: Auxiliary/optional parameters (will use defaults if not provided)
    
    Parameters
    ----------
    template_path : str or Path
        Path to the Mako template file (.mako)
    logger : logging.Logger, optional
        Logger for status messages
        
    Returns
    -------
    tuple (list, list)
        (main_parameters, aux_parameters)
        - main_parameters: List of required parameter names
        - aux_parameters: List of optional parameter names
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    AttributeError
        If template doesn't define get_main_parameters or get_aux_parameters
    Exception
        For other template parsing errors
        
    Examples
    --------
    >>> main, aux = parse_template_parameters('pipe_template.mako')
    >>> print(main)
    ['length', 'diameter']
    >>> print(aux)
    ['m_flow_nominal', 'dp_nominal', 'roughness']
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Check if template file exists
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        raise FileNotFoundError(error_msg)
    
    try:
        # Load the Mako template
        logger.debug(f"Loading template: {template_path}")
        template = Template(filename=str(template_path))
        
        # Extract main (required) parameters
        try:
            main_params_str = template.get_def("get_main_parameters").render()
            main_params = main_params_str.split()
            logger.debug(f"Main parameters found: {main_params}")
        except AttributeError:
            logger.warning(
                f"Template {template_path} does not define 'get_main_parameters'. "
                "Assuming no required parameters."
            )
            main_params = []
        
        # Extract auxiliary (optional) parameters
        try:
            aux_params_str = template.get_def("get_aux_parameters").render()
            aux_params = aux_params_str.split()
            logger.debug(f"Auxiliary parameters found: {aux_params}")
        except AttributeError:
            logger.warning(
                f"Template {template_path} does not define 'get_aux_parameters'. "
                "Assuming no optional parameters."
            )
            aux_params = []
        
        logger.info(
            f"Template parsed successfully: "
            f"{len(main_params)} main params, {len(aux_params)} aux params"
        )
        
        return main_params, aux_params
        
    except Exception as e:
        error_msg = f"Error parsing template {template_path}: {e}"
        logger.error(error_msg)
        raise Exception(error_msg)

In [6]:
parse_template_parameters(r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\pipe\AixLib_Fluid_DistrictHeatingCooling_Pipes_PlugFlowPipeEmbedded.mako")

(['length', 'dIns', 'kIns'],
 ['CPip',
  'VEqu',
  'sta_default',
  'cp_default',
  'C',
  'rho_default',
  'energyDynamics',
  'use_zeta',
  'from_dp',
  'dh',
  'v_nominal',
  'ReC',
  'roughness',
  'm_flow_nominal',
  'm_flow_small',
  'cPip',
  'rhoPip',
  'thickness',
  'T_start_in',
  'T_start_out',
  'initDelay',
  'm_flow_start',
  'R',
  'fac',
  'sum_zetas',
  'linearized',
  'rho_soi',
  'c',
  'thickness_soi',
  'lambda_ground',
  'd_in',
  'nParallel',
  'T0',
  '_m_flow_start',
  '_dp_start',
  'show_T',
  'allowFlowReversal'])

In [9]:

def assign_pipe_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to pipe edges in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. For each edge individually:
       - Check MAIN parameters: in edge → keep, not in edge → try Excel, missing → ERROR
       - Check AUX parameters: in edge → keep, not in edge → try Excel, missing → WARNING
    4. Apply parameters from Excel where needed (never overwrite existing edge attributes)
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object
    template_path : str or Path
        Path to the pipe template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        
    Returns
    -------
    uesgraph : UESGraph
        Updated graph with assigned parameters
    validation_report : dict
        Report containing:
        - 'success': bool, whether all MAIN parameters were found for all edges
        - 'edges_validated': int, number of edges processed
        - 'params_from_graph': int, parameters already in graph
        - 'params_from_excel': int, parameters added from Excel
        - 'missing_main': list of (edge, param) tuples for missing required parameters
        - 'missing_aux': set of parameter names missing across all edges
        - 'errors': list of error messages
        - 'warnings': list of warning messages
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any edge
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    validation_report = {
        'success': True,
        'edges_validated': 0,
        'params_from_graph': 0,
        'params_from_excel': 0,
        'missing_main': [],
        'missing_aux': set(),
        'errors': [],
        'warnings': []
    }
    
    # Step 1: Load template file
    logger.info(f"Loading template file: {template_path}")
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise FileNotFoundError(error_msg)
    
    # Step 2: Parse template and extract parameters
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    # Step 3: Load Excel parameters if provided
    excel_params = {}
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, 'Pipes')
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            validation_report['warnings'].append(warning_msg)
    
    # Step 4: Process each edge individually
    total_edges = len(list(uesgraph.edges()))
    logger.info(f"Processing {total_edges} edges...")
    
    for edge_idx, edge in enumerate(uesgraph.edges(), 1):
        edge_data = uesgraph.edges[edge]
        logger.debug(f"Processing edge {edge_idx}/{total_edges}: {edge}")
        
        # Step 4a: Check and apply MAIN parameters for this edge
        for param in main_parameters:
            if param in edge_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ MAIN '{param}' found in edge")
            elif param in excel_params:
                # Parameter not in edge, but available in Excel - apply it
                edge_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ MAIN '{param}' applied from Excel")
            else:
                # Parameter missing - ERROR!
                validation_report['missing_main'].append((edge, param))
                validation_report['success'] = False
                error_msg = f"Edge {edge}: Missing MAIN parameter '{param}'"
                logger.error(error_msg)
                validation_report['errors'].append(error_msg)
        
        # Step 4b: Check and apply AUX parameters for this edge
        for param in aux_parameters:
            if param in edge_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ AUX '{param}' found in edge")
            elif param in excel_params:
                # Parameter not in edge, but available in Excel - apply it
                edge_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ AUX '{param}' applied from Excel")
            else:
                # Parameter missing - will use Modelica default (collect for summary)
                validation_report['missing_aux'].add(param)
                logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
        
        validation_report['edges_validated'] += 1
    
    # Step 5: Report summary
    logger.info(f"✓ Processed {validation_report['edges_validated']} edges")
    logger.info(f"  - Parameters from graph: {validation_report['params_from_graph']}")
    logger.info(f"  - Parameters from Excel: {validation_report['params_from_excel']}")
    
    # Summary of missing AUX parameters (if any)
    if validation_report['missing_aux']:
        missing_count = len(validation_report['missing_aux'])
        summary_msg = (
            f"⚠ {missing_count} AUX parameter(s) not provided, will use Modelica defaults:\n"
            f"   {', '.join(sorted(validation_report['missing_aux']))}"
        )
        logger.warning(summary_msg)
        validation_report['warnings'].append(summary_msg)
    
    # Check if validation was successful
    if not validation_report['success']:
        error_count = len(validation_report['missing_main'])
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per edge:\n"
        )
        for edge, param in validation_report['missing_main']:
            error_msg += f"  - Edge {edge}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per edge: add to uesgraph edge attributes\n"
            f"  → If parameter is same for all edges: add to Excel sheet 'pipes'"
        )
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")
    
    return uesgraph, validation_report

In [10]:
graph2, report = assign_pipe_parameters(graph, r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\pipe\AixLib_Fluid_DistrictHeatingCooling_Pipes_PlugFlowPipeEmbedded.mako", r"C:\Users\rka-lko\Downloads\uesgraphs_parameters_template.xlsx")

⚠ 34 AUX parameter(s) not provided, will use Modelica defaults:
   C, CPip, R, ReC, T0, T_start_in, T_start_out, VEqu, _dp_start, _m_flow_start, c, cPip, cp_default, d_in, dh, energyDynamics, from_dp, initDelay, lambda_ground, linearized, m_flow_nominal, m_flow_small, m_flow_start, nParallel, rhoPip, rho_default, rho_soi, show_T, sta_default, sum_zetas, thickness, thickness_soi, use_zeta, v_nominal


In [11]:
graph.edges(data=True)

EdgeDataView([(1002, 1010, {'attr_dict': {'id': 9, 'DN': 65, 'length': 61.98113731669513}, 'diameter': 0.0697, 'dIns': 0.1, 'length': 62.67448664094058, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1002, 1014, {'attr_dict': {'id': 13, 'DN': 200, 'length': 40.10060652921126}, 'diameter': 0.2101, 'dIns': 0.1, 'length': 40.10060652984215, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1002, 1025, {'attr_dict': {'id': 2, 'DN': 200, 'length': 23.793988913605595}, 'diameter': 0.2101, 'length': 23.793988913832845, 'dIns': 0.04, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1004, 1006, {'attr_dict': {'id': 5, 'DN': 125, 'length': 23.152279454397373}, 'diameter': 0.1325, 'dIns': 0.1, 'length': 23.15227945754485, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1004, 1007, {'attr_dict': {'id': 7, 'DN': 125, 'length': 50.66733186046747}, 'diameter': 0.1325, 'dIns': 0.1, 'length': 67.213436

In [12]:

def assign_supply_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to supply nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. For each supply node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    4. Apply parameters from Excel where needed (never overwrite existing node attributes)
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object
    template_path : str or Path
        Path to the supply template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        
    Returns
    -------
    uesgraph : UESGraph
        Updated graph with assigned parameters
    validation_report : dict
        Report containing:
        - 'success': bool, whether all MAIN parameters were found for all nodes
        - 'nodes_validated': int, number of supply nodes processed
        - 'params_from_graph': int, parameters already in graph
        - 'params_from_excel': int, parameters added from Excel
        - 'missing_main': list of (node, param) tuples for missing required parameters
        - 'missing_aux': set of parameter names missing across all nodes
        - 'errors': list of error messages
        - 'warnings': list of warning messages
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any supply node
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    validation_report = {
        'success': True,
        'nodes_validated': 0,
        'params_from_graph': 0,
        'params_from_excel': 0,
        'missing_main': [],
        'missing_aux': set(),
        'errors': [],
        'warnings': []
    }
    
    # Step 1: Load template file
    logger.info(f"Loading template file: {template_path}")
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise FileNotFoundError(error_msg)
    
    # Step 2: Parse template and extract parameters
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    # Step 3: Load Excel parameters if provided
    excel_params = {}
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, 'Supply')
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            validation_report['warnings'].append(warning_msg)
    
    # Step 4: Find supply nodes
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    supply_nodes = []
    for node in uesgraph.nodelist_building:
        if uesgraph.nodes[node].get(is_supply_key, False):
            supply_nodes.append(node)
    
    if not supply_nodes:
        warning_msg = f"No supply nodes found (looking for '{is_supply_key}' = True)"
        logger.warning(warning_msg)
        validation_report['warnings'].append(warning_msg)
        return uesgraph, validation_report
    
    # Step 5: Process each supply node individually
    total_nodes = len(supply_nodes)
    logger.info(f"Processing {total_nodes} supply node(s)...")
    
    for node_idx, node in enumerate(supply_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing supply node {node_idx}/{total_nodes}: {node}")
        
        # Step 5a: Check and apply MAIN parameters for this node
        for param in main_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ MAIN '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but available in Excel - apply it
                node_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ MAIN '{param}' applied from Excel")
            else:
                # Parameter missing - ERROR!
                validation_report['missing_main'].append((node, param))
                validation_report['success'] = False
                error_msg = f"Node {node}: Missing MAIN parameter '{param}'"
                logger.error(error_msg)
                validation_report['errors'].append(error_msg)
        
        # Step 5b: Check and apply AUX parameters for this node
        for param in aux_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ AUX '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but available in Excel - apply it
                node_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ AUX '{param}' applied from Excel")
            else:
                # Parameter missing - will use Modelica default (collect for summary)
                validation_report['missing_aux'].add(param)
                logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
        
        validation_report['nodes_validated'] += 1
    
    # Step 6: Report summary
    logger.info(f"✓ Processed {validation_report['nodes_validated']} supply node(s)")
    logger.info(f"  - Parameters from graph: {validation_report['params_from_graph']}")
    logger.info(f"  - Parameters from Excel: {validation_report['params_from_excel']}")
    
    # Summary of missing AUX parameters (if any)
    if validation_report['missing_aux']:
        missing_count = len(validation_report['missing_aux'])
        summary_msg = (
            f"⚠ {missing_count} AUX parameter(s) not provided, will use Modelica defaults:\n"
            f"   {', '.join(sorted(validation_report['missing_aux']))}"
        )
        logger.warning(summary_msg)
        validation_report['warnings'].append(summary_msg)
    
    # Check if validation was successful
    if not validation_report['success']:
        error_count = len(validation_report['missing_main'])
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per node:\n"
        )
        for node, param in validation_report['missing_main']:
            error_msg += f"  - Node {node}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per node: add to uesgraph node attributes\n"
            f"  → If parameter is same for all supply nodes: add to Excel sheet 'Supply'"
        )
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")
    
    return uesgraph, validation_report

In [14]:
assign_supply_parameters(graph2, r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\supply\AixLib_Fluid_DistrictHeatingCooling_Supplies_OpenLoop_SourceIdeal.mako", r"C:\Users\rka-lko\Downloads\uesgraphs_parameters_template.xlsx")

Node 1025: Missing MAIN parameter 'pReturn'
Node 1025: Missing MAIN parameter 'TReturn'


Validation FAILED: 2 missing MAIN parameter(s)
Missing parameters per node:
  - Node 1025: 'pReturn'
  - Node 1025: 'TReturn'

Fix suggestions:
  → If parameter varies per node: add to uesgraph node attributes
  → If parameter is same for all supply nodes: add to Excel sheet 'Supply'


ValueError: Validation FAILED: 2 missing MAIN parameter(s)
Missing parameters per node:
  - Node 1025: 'pReturn'
  - Node 1025: 'TReturn'

Fix suggestions:
  → If parameter varies per node: add to uesgraph node attributes
  → If parameter is same for all supply nodes: add to Excel sheet 'Supply'

In [16]:

def assign_demand_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to demand nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. For each demand node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    4. Apply parameters from Excel where needed (never overwrite existing node attributes)
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object
    template_path : str or Path
        Path to the demand template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        
    Returns
    -------
    uesgraph : UESGraph
        Updated graph with assigned parameters
    validation_report : dict
        Report containing:
        - 'success': bool, whether all MAIN parameters were found for all nodes
        - 'nodes_validated': int, number of demand nodes processed
        - 'params_from_graph': int, parameters already in graph
        - 'params_from_excel': int, parameters added from Excel
        - 'missing_main': list of (node, param) tuples for missing required parameters
        - 'missing_aux': set of parameter names missing across all nodes
        - 'errors': list of error messages
        - 'warnings': list of warning messages
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any demand node
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    validation_report = {
        'success': True,
        'nodes_validated': 0,
        'params_from_graph': 0,
        'params_from_excel': 0,
        'missing_main': [],
        'missing_aux': set(),
        'errors': [],
        'warnings': []
    }
    
    # Step 1: Load template file
    logger.info(f"Loading template file: {template_path}")
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise FileNotFoundError(error_msg)
    
    # Step 2: Parse template and extract parameters
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    # Step 3: Load Excel parameters if provided
    excel_params = {}
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, 'Demands')
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            validation_report['warnings'].append(warning_msg)
    
    # Step 4: Find demand nodes (buildings that are NOT supply)
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    demand_nodes = []
    for node in uesgraph.nodelist_building:
        if not uesgraph.nodes[node].get(is_supply_key, False):
            demand_nodes.append(node)
    
    if not demand_nodes:
        warning_msg = f"No demand nodes found (looking for buildings with '{is_supply_key}' = False)"
        logger.warning(warning_msg)
        validation_report['warnings'].append(warning_msg)
        return uesgraph, validation_report
    
    # Step 5: Process each demand node individually
    total_nodes = len(demand_nodes)
    logger.info(f"Processing {total_nodes} demand node(s)...")
    
    for node_idx, node in enumerate(demand_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing demand node {node_idx}/{total_nodes}: {node}")
        
        # Step 5a: Check and apply MAIN parameters for this node
        for param in main_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ MAIN '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but available in Excel - apply it
                node_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ MAIN '{param}' applied from Excel")
            else:
                # Parameter missing - ERROR!
                validation_report['missing_main'].append((node, param))
                validation_report['success'] = False
                error_msg = f"Node {node}: Missing MAIN parameter '{param}'"
                logger.error(error_msg)
                validation_report['errors'].append(error_msg)
        
        # Step 5b: Check and apply AUX parameters for this node
        for param in aux_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ AUX '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but available in Excel - apply it
                node_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ AUX '{param}' applied from Excel")
            else:
                # Parameter missing - will use Modelica default (collect for summary)
                validation_report['missing_aux'].add(param)
                logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
        
        validation_report['nodes_validated'] += 1
    
    # Step 6: Report summary
    logger.info(f"✓ Processed {validation_report['nodes_validated']} demand node(s)")
    logger.info(f"  - Parameters from graph: {validation_report['params_from_graph']}")
    logger.info(f"  - Parameters from Excel: {validation_report['params_from_excel']}")
    
    # Summary of missing AUX parameters (if any)
    if validation_report['missing_aux']:
        missing_count = len(validation_report['missing_aux'])
        summary_msg = (
            f"⚠ {missing_count} AUX parameter(s) not provided, will use Modelica defaults:\n"
            f"   {', '.join(sorted(validation_report['missing_aux']))}"
        )
        logger.warning(summary_msg)
        validation_report['warnings'].append(summary_msg)
    
    # Check if validation was successful
    if not validation_report['success']:
        error_count = len(validation_report['missing_main'])
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per node:\n"
        )
        for node, param in validation_report['missing_main']:
            error_msg += f"  - Node {node}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per node: add to uesgraph node attributes\n"
            f"  → If parameter is same for all demand nodes: add to Excel sheet 'Demands'"
        )
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")
    
    return uesgraph, validation_report

In [17]:
assign_demand_parameters(graph2, r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\demand\AixLib_Fluid_DistrictHeatingCooling_Demands_ClosedLoop_DHCSubstationHeatPumpChiller.mako", r"C:\Users\rka-lko\Downloads\uesgraphs_parameters_template.xlsx")

Node 1026: Missing MAIN parameter 'heaDem_max'
Node 1026: Missing MAIN parameter 'cooDem_max'
Node 1026: Missing MAIN parameter 'deltaT_heaSecSet'
Node 1026: Missing MAIN parameter 'deltaT_cooSecSet'
Node 1026: Missing MAIN parameter 'deltaT_heaPriSet'
Node 1026: Missing MAIN parameter 'deltaT_cooPriSet'
Node 1027: Missing MAIN parameter 'heaDem_max'
Node 1027: Missing MAIN parameter 'cooDem_max'
Node 1027: Missing MAIN parameter 'deltaT_heaSecSet'
Node 1027: Missing MAIN parameter 'deltaT_cooSecSet'
Node 1027: Missing MAIN parameter 'deltaT_heaPriSet'
Node 1027: Missing MAIN parameter 'deltaT_cooPriSet'
Node 1028: Missing MAIN parameter 'heaDem_max'
Node 1028: Missing MAIN parameter 'cooDem_max'
Node 1028: Missing MAIN parameter 'deltaT_heaSecSet'
Node 1028: Missing MAIN parameter 'deltaT_cooSecSet'
Node 1028: Missing MAIN parameter 'deltaT_heaPriSet'
Node 1028: Missing MAIN parameter 'deltaT_cooPriSet'
Node 1029: Missing MAIN parameter 'heaDem_max'
Node 1029: Missing MAIN parameter '

ValueError: Validation FAILED: 24 missing MAIN parameter(s)
Missing parameters per node:
  - Node 1026: 'heaDem_max'
  - Node 1026: 'cooDem_max'
  - Node 1026: 'deltaT_heaSecSet'
  - Node 1026: 'deltaT_cooSecSet'
  - Node 1026: 'deltaT_heaPriSet'
  - Node 1026: 'deltaT_cooPriSet'
  - Node 1027: 'heaDem_max'
  - Node 1027: 'cooDem_max'
  - Node 1027: 'deltaT_heaSecSet'
  - Node 1027: 'deltaT_cooSecSet'
  - Node 1027: 'deltaT_heaPriSet'
  - Node 1027: 'deltaT_cooPriSet'
  - Node 1028: 'heaDem_max'
  - Node 1028: 'cooDem_max'
  - Node 1028: 'deltaT_heaSecSet'
  - Node 1028: 'deltaT_cooSecSet'
  - Node 1028: 'deltaT_heaPriSet'
  - Node 1028: 'deltaT_cooPriSet'
  - Node 1029: 'heaDem_max'
  - Node 1029: 'cooDem_max'
  - Node 1029: 'deltaT_heaSecSet'
  - Node 1029: 'deltaT_cooSecSet'
  - Node 1029: 'deltaT_heaPriSet'
  - Node 1029: 'deltaT_cooPriSet'

Fix suggestions:
  → If parameter varies per node: add to uesgraph node attributes
  → If parameter is same for all demand nodes: add to Excel